In [1]:
#load all modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("H&MDatasetFinal1.csv")

In [3]:
display(df.head())

,customer_id,price_x,sales_channel_id,age,fashion_news_frequency,product_group_name_Accessories,product_group_name_Bags,product_group_name_Cosmetic,product_group_name_Furniture,product_group_name_Garment Full body,product_group_name_Garment Lower body,product_group_name_Garment Upper body,product_group_name_Interior textile,product_group_name_Items,product_group_name_Nightwear,product_group_name_Shoes,product_group_name_Socks & Tights,product_group_name_Swimwear,product_group_name_Underwear,product_group_name_Underwear/nightwear,product_group_name_Unknown,perceived_colour_master_name_Beige,perceived_colour_master_name_Black,perceived_colour_master_name_Blue,perceived_colour_master_name_Bluish Green,perceived_colour_master_name_Brown,perceived_colour_master_name_Green,perceived_colour_master_name_Grey,perceived_colour_master_name_Khaki green,perceived_colour_master_name_Lilac Purple,perceived_colour_master_name_Metal,perceived_colour_master_name_Mole,perceived_colour_master_name_Orange,perceived_colour_master_name_Pink,perceived_colour_master_name_Red,perceived_colour_master_name_Turquoise,perceived_colour_master_name_Unknown,perceived_colour_master_name_White,perceived_colour_master_name_Yellow,perceived_colour_master_name_Yellowish Green,perceived_colour_master_name_undefined,department_name_AK Bottoms,department_name_AK Dresses & Outdoor,department_name_AK Other,department_name_AK Tops Jersey & Woven,department_name_AK Tops Knitwear,department_name_Accessories,department_name_Accessories Boys,department_name_Asia Assortment,department_name_Baby Boy Jersey Fancy,department_name_Baby Boy Knitwear,department_name_Baby Boy Outdoor,department_name_Baby Boy Woven,department_name_Baby Exclusive,department_name_Baby Girl Jersey Fancy,department_name_Baby Girl Knitwear,department_name_Baby Girl Outdoor,department_name_Baby Girl Woven,department_name_Baby Nightwear,department_name_Baby Shoes,department_name_Baby Socks,department_name_Baby Toys/Acc,department_name_Baby basics,department_name_Bags,department_name_Bags & Items,department_name_Basic 1,department_name_Basics,department_name_Belts,department_name_Blazer,department_name_Blazer S&T,department_name_Blouse,department_name_Blouse & Dress,department_name_Boots,department_name_Bottoms,department_name_Bottoms Boys,department_name_Bottoms Girls,department_name_Boys Local Relevance,department_name_Boys Small Acc & Bags,department_name_Campaigns,department_name_Casual Lingerie,department_name_Clean Lingerie,department_name_Conscious Exclusive,department_name_Denim Other Garments,department_name_Denim Trousers,department_name_Denim shorts,department_name_Denim trousers,department_name_Denim wardrobe H&M man inactive from S.6,department_name_Divided Shoes,department_name_Divided Swimwear,department_name_Divided+,department_name_Divided+ inactive from s.1,department_name_Dress,department_name_Dress-up Boys,department_name_Dresses,department_name_Dresses DS,department_name_EQ & Special Collections,department_name_EQ Divided Basics,department_name_EQ H&M Man,department_name_Equatorial,department_name_Equatorial Assortment,department_name_Everyday Waredrobe Denim,department_name_Expressive Lingerie,department_name_Flats,department_name_Functional Lingerie,department_name_Girls Local Relevance,department_name_Girls Projects,department_name_Girls Small Acc/Bags,department_name_Gloves/Hats,department_name_Hair Accessories,department_name_Heavy Basic Jersey,department_name_Heels,department_name_Jacket,department_name_Jacket Casual,department_name_Jacket Smart,department_name_Jacket Street,department_name_Jackets,department_name_Jersey,department_name_Jersey Basic,department_name_Jersey Fancy,department_name_Jersey Fancy DS,department_name_Jersey License,department_name_Jersey Occasion,department_name_Jersey fancy,department_name_Jersey inactive from S.6,department_name_Jersey inactive from s1,department_name_Jersey/Knitwear Premium,department_name_Jewellery,department_name_Jewellery Extended,de

* From our previous research, we find customers that made purchases in both 2019 and 2020 and identified those high value customers in 2020 based on their whole year sales. Due to the large volume of the dataset, here we focus on these high-value customers first, trying to provide recommendations for these customer based on their purchase history
* HV_customer_list stores all high-value customer_id.

In [4]:
HV_customer_list = list(df[df['Y']==2].customer_id.value_counts().keys())
print(len(HV_customer_list))  #208372

208372


In [5]:
transaction_data = pd.read_csv("transactions_train.csv/transactions_train.csv")

In [6]:
transaction_data.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [7]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [8]:
transaction_data["t_dat"] = pd.to_datetime(transaction_data["t_dat"], format = "%Y-%m-%d") 

Here we specify the problem, we select those high-value customers purchases in May 2020, and use our algorithm to provide 5 recommended items. 

To test the accuracy of our predictions, we look into the purchases of these customers in next 3 months -- June, July, August, and see if customers bought what we recommended. The score here are the number of items that are purchased in our prediction basket.

For example, if we recommend item A, B, C, D, E to a specific customer, and three of them are bought by this customer, the score for this is 3.


In [9]:
pred_set = transaction_data[(transaction_data['t_dat']>='2020-05-01') & (transaction_data['t_dat']<='2020-05-31')]
test_set = transaction_data[(transaction_data['t_dat']>='2020-06-01') & (transaction_data['t_dat']<='2020-08-31')]


In [10]:
#print(len(pred_set))  1361815
pred_set = pred_set[pred_set['customer_id'].isin(HV_customer_list)]
print(len(pred_set)) #785634
test_set = test_set[test_set['customer_id'].isin(HV_customer_list)]
print(len(test_set)) #2467377

785634
2467377


In [11]:
pred_df = pred_set[['customer_id','article_id']]
test_df = test_set[['customer_id','article_id']]

#### Item recommendation based on item similarity

- Method 1: find similar products to what a customer purchased previously
Similarity here is based on text analysis.

In [12]:
#use text analysis techniques to measure similarity between items
articles = pd.read_csv("articles.csv/articles.csv")
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,3,Light,9,White,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


Information about the product forms a product "document" for each article:

* prod_name: A unique identifier of every product and its name (not the same).
* product_type_name: group of the product_name
* product_group_name
* graphical_appearance_name: The group of graphics and its name
* colour_group_name
* perceived_colour_value_name & perceived_colour_master_name: color info
* department_name
* index_name & index_group_name
* section_name
* garment_group_name
* detail_desc

In [13]:
articles.isnull().sum()

article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     416
dtype: int64

In [14]:
print(articles.shape) #(105542, 25)
info_columns = [
    'article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name','perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name','index_group_name', 'section_name','garment_group_name', 'detail_desc'
]
articles_info = articles[info_columns]

(105542, 25)


In [15]:
for i in articles_info.columns[1:]:
    articles_info[i] = articles_info[i].str.replace(" ","")

In [16]:
articles_info['info'] = articles_info[info_columns[1:]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
articles_df = articles_info[['article_id','info']]

In [17]:
articles_df.head()

,article_id,info
0,108775015,Straptop Vesttop GarmentUpperbody Solid Black ...
1,108775044,Straptop Vesttop GarmentUpperbody Solid White ...
2,108775051,Straptop(1) Vesttop GarmentUpperbody Stripe Of...
3,110065001,OPT-shirt(Idro) Bra Underwear Solid Black Dark...
4,110065002,OPT-shirt(Idro) Bra Underwear Solid White Ligh...


In [18]:
## find top5 similar products for articles
#to simplify the process, here we just focus on 8000 items
# but this method can be extended to the whole dataset.
articles_subset = articles_df[:8000]
#print(articles_subset['article_id'].duplicated().sum())  0
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector = TfidfVectorizer(use_idf=True,stop_words='english',max_df=0.8)
X = tfidf_vector.fit_transform(articles_subset['info'])
print(X.shape)

(8000, 8421)


In [19]:
#compute the matrix similarity to find similar article
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(X, X)

In [20]:
indices = pd.Series(articles_subset.index, index=articles_subset['article_id'])
def get_similar_articles(article_id, cosine_sim):
    idx = indices[article_id]
    #get the similarity scores
    sim_scores =  list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #get the scores of the top 5 similar articles apart from itself
    sim_scores = sim_scores[1:6]
    articles_indices = [i[0] for i in sim_scores]
    return articles_subset['article_id'].iloc[articles_indices]

In [21]:
#use the function above to give reommendation to each customer's transaction data
#Note here we only have the similar products for 8000 articles, so we temporarily could only give 
#recommendation on these items and test the accuracy for these customers
pred_df2= pred_df[pred_df['article_id'].isin(articles_subset['article_id'])]
print(pred_df2.shape)
test_df2 = test_df[test_df['customer_id'].isin(pred_df2['customer_id'])]
print(test_df2.shape)

(29892, 2)
(286982, 2)


In [22]:
pred_df2['similar_articles'] = pred_df2['article_id'].apply(lambda x:list(get_similar_articles(x,cosine_sim=cosine_sim)))


In [56]:
pred_df2[['article_id','similar_articles']][100:105]

,article_id,similar_articles
25280017,413029008,"[413029003, 448515023, 448515033, 400285011, 4..."
25280065,501722031,"[501722030, 501722025, 501722029, 501722022, 5..."
25280068,501722030,"[501722031, 501722025, 501722017, 501722005, 5..."
25280074,445400029,"[445400025, 445400022, 445400014, 445400013, 4..."
25280121,399201042,"[399201042, 399201025, 399201045, 399201035, 3..."


In [54]:
for i in pred_df2['similar_articles'][:5]:
    print(i)

[349301045, 349301046, 349301028, 349301029, 349301025]
[408875030, 408875016, 408875009, 408875028, 408875010]
[351484026, 351484009, 351484002, 351484013, 351484034]
[351484026, 351484009, 351484002, 351484013, 351484034]
[470789031, 470789001, 470789028, 470789019, 470789012]


In [57]:
for i in pred_df2['similar_articles'][100:105]:
    print(i)

[413029003, 448515023, 448515033, 400285011, 448515027]
[501722030, 501722025, 501722029, 501722022, 501722024]
[501722031, 501722025, 501722017, 501722005, 501722020]
[445400025, 445400022, 445400014, 445400013, 445400024]
[399201042, 399201025, 399201045, 399201035, 399201002]


In [24]:
def app_func(dataf):
  temp = []
  dataf.reset_index(inplace=True,drop=True)
  for i in range(dataf.shape[0]):
    #print(i)
    #print(i,dataf['similar_articles'][i])
    temp = temp + dataf['similar_articles'][i]
  #print('temp',temp)
  return temp#[ item for elem in temp for item in elem]

In [25]:
fin = pd.DataFrame(pred_df2.groupby(['customer_id']).apply(app_func))
fin = fin.reset_index()

In [26]:
fin.columns = ['customer_id','predictions']

In [27]:
#in case there are duplicates of recommended articles, we do a processing to drop potential duplicates
for i in range(fin.shape[0]):
    fin['predictions'][i] = list(set(fin['predictions'][i]))

In [28]:
fin.head()

,customer_id,predictions
0,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,"[452818021, 452818029, 452818031, 452818034, 4..."
1,00080403a669b3b89d1bef1ec73ea466d95e39698d6dde...,"[496923008, 451744001, 451744005, 451744006, 4..."
2,0008d644deb96bdc0ca262f161cf6d5e9a4e619bb75faa...,"[486639009, 486639011, 486639012, 486639013, 4..."
3,000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060...,"[488561027, 488561032, 488561036, 488561015, 4..."
4,000a74a8ce6e616c1d26d186d8ffae487878b89223b60f...,"[408875009, 408875016, 408875028, 408875030, 4..."


Therefore, we get several articles that are predicted to be bought by every customer. Note here we have different number of predictions for different customers. 

In order to make comparison, we calculate the percentage of right predictions.

Now we calculate the percentage of right predictions for each customer.


In [29]:
#let's use the following three months transaction data to test our predictions
actual_articles = test_df2.groupby('customer_id')['article_id'].apply(list).reset_index(name='articles_list')
actual_articles.head()

,customer_id,articles_list
0,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,"[861608003, 875469004, 733097008]"
1,00080403a669b3b89d1bef1ec73ea466d95e39698d6dde...,"[832505001, 902992001, 556260001, 579541072, 8..."
2,0008d644deb96bdc0ca262f161cf6d5e9a4e619bb75faa...,"[869424001, 855262002]"
3,000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060...,"[850906001, 690513007, 716545008, 833484002, 8..."
4,000c5c714aefd0d5ed1205e2781070167826ffc117ab9e...,"[579541072, 869691001, 579541001, 901955001, 8..."


In [30]:
print(fin.shape, actual_articles.shape) #(19477, 2) (17430, 2)
comparison = pd.merge(fin, actual_articles, on='customer_id', how='left')
print(comparison.shape)
print(comparison['articles_list'].isnull().sum()) #2047

(19477, 2) (17430, 2)
(19477, 3)
2047


In [60]:
comparison[comparison['accuracy']!=0].head()

,customer_id,predictions,articles_list,accuracy
54,00b6efd6ae233e446805ced7ad1f6003afac91e4b42c37...,"[253448001, 253448003, 253448056, 253448059, 2...","[689389048, 596740012, 253448059, 253448003, 5...",0.400000
92,012ae734fb97c1f44025243d4b5a33ebfad7c222a80cf9...,"[372860001, 373506018, 373506019, 372860069, 3...","[372860001, 857572001, 464297007, 891655001, 8...",0.066667
99,013c3aadf29d43aae7c100766aa1f20b2730fb897faa64...,"[372860001, 253448001, 253448003, 253448002, 2...","[253448003, 762846007, 856617003, 824764004, 5...",0.090909
186,028a493371e362fa06b6a446244e46801c9123379a861b...,"[372860002, 372860043, 160442007, 160442042, 1...","[821886001, 860632003, 160442043, 841383003, 2...",0.200000
191,02aae8afcbc7874b1457056ee3d7999776363967a79564...,"[372860002, 372860043, 160442007, 160442042, 1...","[806388009, 783346001, 156224001, 349301001, 8...",0.200000


In [32]:
accuracy_list = []
for i in range(comparison.shape[0]):
    if isinstance(comparison['articles_list'][i], list)==False:
        accuracy=0
    else:
        list1 =  list(comparison['predictions'][i])
        list2 = list(comparison['articles_list'][i])
        accuracy = len([i for i in list1 if i in list2])/len(list1)
    accuracy_list.append(accuracy)

comparison['accuracy'] = np.array(accuracy_list)

In [33]:
from collections import Counter
counter = Counter(comparison['accuracy'].value_counts())

In [58]:
len(comparison)

19477

In [80]:
print(len(comparison[comparison['accuracy']!=0])/len(comparison))
print(len(comparison[comparison['accuracy']!=0]))

0.03691533603737742
719


In [50]:
correct = 0
incorrect= 0
for i in range(comparison.shape[0]):
    if isinstance(comparison['articles_list'][i], list)==False:
        incorrect+=len(list(comparison['predictions'][i]))
    else:
        for article in list(comparison['predictions'][i]):
            if article in list(comparison['articles_list'][i]):
                correct+=1
            else:
                incorrect+=1
    

print(correct)
print(incorrect)

815
122507


Unfortunately, here the accuracy is quite low, which indicates that our predictions are not quite satisfying for customers. 

From my perspective, I think this could be the following reason:
- Here we only select a subset of 8000 articles to calculate 
- Maybe one-month data is not enough to capture 
Try to have a look at longer term, to see if these predicted items are bought.

In [36]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        int64         
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.2+ GB


In [37]:
pred_set2 = transaction_data[(transaction_data['t_dat']>='2020-04-01') & (transaction_data['t_dat']<='2020-05-31')]
test_set2 = transaction_data[(transaction_data['t_dat']>='2020-06-01') & (transaction_data['t_dat']<='2020-09-30')]

In [39]:
pred_set2 = pred_set2[pred_set2['customer_id'].isin(HV_customer_list)]
print(len(pred_set2)) #1576417
test_set2 = test_set2[test_set2['customer_id'].isin(HV_customer_list)]
print(len(test_set2)) #2928357

pred_df3 = pred_set2[['customer_id','article_id']]
test_df3 = test_set2[['customer_id','article_id']]

1576417
2928357


In [40]:
pred_df4= pred_df3[pred_df3['article_id'].isin(articles_subset['article_id'])]
print(pred_df4.shape)
test_df4 = test_df3[test_df3['customer_id'].isin(pred_df4['customer_id'])]
print(test_df4.shape)

(60516, 2)
(610503, 2)


In [41]:
pred_df4['similar_articles'] = pred_df4['article_id'].apply(lambda x:list(get_similar_articles(x,cosine_sim=cosine_sim)))
pred_df4.head()

,customer_id,article_id,similar_articles
23934176,004fb527dcbefecd7cbd299f9aed3276a6ff2d2639da6f...,241412052,"[241412046, 241412045, 241412043, 241412044, 2..."
23934178,004fb527dcbefecd7cbd299f9aed3276a6ff2d2639da6f...,316085001,"[316085002, 316085013, 377277001, 516712001, 3..."
23934182,0055692692c0cdd5278610f4b62ecabc8130eff743170d...,448509025,"[448509024, 448509026, 448509009, 448509018, 4..."
23934183,0064cd1ee810d4caabd1182a8f177479b82b18961bd76b...,270375006,"[270375001, 270375005, 270375004, 215303002, 2..."
23934261,00c2a879ad78b87eea05d519dc0be1a0b8382ef3643ca8...,278811006,"[278811009, 278811011, 278811002, 278811010, 2..."


In [42]:
fin2 = pd.DataFrame(pred_df4.groupby(['customer_id']).apply(app_func))
fin2 = fin2.reset_index()

fin2.columns = ['customer_id','predictions']
for i in range(fin2.shape[0]):
    fin2['predictions'][i] = list(set(fin2['predictions'][i]))

fin2.head()

,customer_id,predictions
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[351484034, 351484039, 351484009, 351484013, 3..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[349301025, 349301028, 349301029, 349301045, 3..."
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[337991009, 337991010, 337991001, 337991005, 3..."
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[278811009, 278811011, 278811002, 278811006, 2..."
4,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,"[452818021, 452818029, 452818031, 452818034, 4..."


In [43]:
actual_articles2 = test_df4.groupby('customer_id')['article_id'].apply(list).reset_index(name='articles_list')
actual_articles2.head()

,customer_id,articles_list
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,[826211002]
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[848460002, 848460002, 848460002, 852746001, 8..."
2,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[842792001, 859118002, 886540001, 869424001, 6..."
3,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,"[861608003, 875469004, 733097008]"
4,00080403a669b3b89d1bef1ec73ea466d95e39698d6dde...,"[832505001, 902992001, 556260001, 579541072, 8..."


In [44]:
print(fin2.shape, actual_articles2.shape) #(19477, 2) (17430, 2)
comparison2 = pd.merge(fin2, actual_articles2, on='customer_id', how='left')
print(comparison2.shape)
print(comparison2['articles_list'].isnull().sum()) #2047

comparison2.head()

(35672, 2) (32371, 2)
(35672, 3)
3301


,customer_id,predictions,articles_list
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[351484034, 351484039, 351484009, 351484013, 3...",[826211002]
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[349301025, 349301028, 349301029, 349301045, 3...",NaN
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[337991009, 337991010, 337991001, 337991005, 3...","[848460002, 848460002, 848460002, 852746001, 8..."
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[278811009, 278811011, 278811002, 278811006, 2...","[842792001, 859118002, 886540001, 869424001, 6..."
4,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,"[452818021, 452818029, 452818031, 452818034, 4...","[861608003, 875469004, 733097008]"


In [45]:
accuracy_list2 = []
for i in range(comparison2.shape[0]):
    if isinstance(comparison2['articles_list'][i], list)==False:
        accuracy=0
    else:
        list1 =  list(comparison2['predictions'][i])
        list2 = list(comparison2['articles_list'][i])
        accuracy = len([i for i in list1 if i in list2])/len(list1)
    accuracy_list2.append(accuracy)

comparison2['accuracy'] = np.array(accuracy_list2)

In [46]:
print(len(comparison[comparison['accuracy']!=0])/len(comparison))
print(len(comparison[comparison['accuracy']!=0]))

0.03691533603737742
719


In [47]:
#store the datasets
pred_set.to_csv("pred_set.csv")
test_set.to_csv("pred_set.csv")
pred_set2.to_csv("pred_set.csv")
test_set2.to_csv("pred_set.csv")

pred_df2.to_csv("pred_df2.csv")
test_df2.to_csv("test_df2.csv")

fin.to_csv("fin.csv")
fin2.to_csv("fin2.csv")
actual_articles.to_csv("actual_articles.csv")
actual_articles2.to_csv("actual_articles2.csv")

comparison.to_csv("comparison.csv")
comparison2.to_csv("comparison2.csv")

In [29]:
'''
Code for generating the similar result to what is in the submission file.
t = fin[:20]
for i in range(t.shape[0]):
  t['predictions'][i] = ' '.join(['0'+ str(x) for x in t['predictions'][i]])
'''
